In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Directorio raíz del dataset en Google Drive
root_dir = '/content/drive/MyDrive/SIS421/2Parcial/datasetbasura/'


In [15]:
transform = transforms.Compose([
    transforms.Resize((800, 600)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from google.colab import drive
import os
import json
from PIL import Image
from sklearn.model_selection import train_test_split

class TrashDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


# Cargar imágenes y etiquetas
image_paths = []
labels = []

class_names = os.listdir(root_dir)
for i, class_name in enumerate(class_names):
    class_dir = os.path.join(root_dir, class_name)
    labels_file = os.path.join(class_dir, class_name + ".json")
    
    with open(labels_file, "r") as f:
        annotations = json.load(f)

    for annotation in annotations:
        filename = annotation["annotation"]["filename"]
        image_path = os.path.join(class_dir, filename)
        image_paths.append(image_path)
        labels.append(class_name)

        

# Dividir las imágenes y las etiquetas en conjuntos de entrenamiento y prueba
train_images, test_images, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

# Creación del dataset de entrenamiento
train_dataset = TrashDataset(train_images, train_labels, transform=transform)

# Creación del dataset de prueba
test_dataset = TrashDataset(test_images, test_labels, transform=transform)

# Crear el diccionario 'dataset'
dataset = {
    'train': train_dataset,
    'test': test_dataset
}

# Crear el DataLoader para el conjunto de entrenamiento
train_dataloader = DataLoader(dataset['train'], batch_size=16, shuffle=True, pin_memory=True)

# Crear el DataLoader para el conjunto de prueba
test_dataloader = DataLoader(dataset['test'], batch_size=32, pin_memory=True)

# Crear el diccionario 'dataloader'
dataloader = {
    'train': train_dataloader,
    'test': test_dataloader
}


# Verificar la cantidad de imágenes en cada conjunto
print("Número de imágenes de entrenamiento:", len(dataset['train']))
print("Número de imágenes de prueba:", len(dataset['test']))


NameError: ignored

In [24]:
# Obtener un lote de datos de entrenamiento
imgs, labels = next(iter(dataloader['train']))
print("Tamaño de las imágenes:", type(imgs))
#labels = torch.tensor(label)  # Convertir las etiquetas a tensores
print("Tamaño de las etiquetas:", type(labels))
for i, label in enumerate(labels):
    print("Etiqueta", i+1, ":", label)

Tamaño de las imágenes: <class 'torch.Tensor'>
Tamaño de las etiquetas: <class 'tuple'>
Etiqueta 1 : papel
Etiqueta 2 : carton
Etiqueta 3 : carton
Etiqueta 4 : papel
Etiqueta 5 : carton
Etiqueta 6 : carton
Etiqueta 7 : papel
Etiqueta 8 : papel
Etiqueta 9 : papel
Etiqueta 10 : papel
Etiqueta 11 : papel
Etiqueta 12 : papel
Etiqueta 13 : papel
Etiqueta 14 : papel
Etiqueta 15 : bolsa
Etiqueta 16 : carton


Número de imágenes de entrenamiento: 25804
Número de imágenes de validación: 6452


In [25]:
import torchvision
model=torchvision.models. regnet_x_400mf(pretrained=True)
#print(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_400MF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/regnet_x_400mf-adf1edd5.pth" to /root/.cache/torch/hub/checkpoints/regnet_x_400mf-adf1edd5.pth
100%|██████████| 21.3M/21.3M [00:00<00:00, 55.8MB/s]


In [26]:
num_classes = 3 # Número de clases de basura
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [35]:
def train(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        accuracy = evaluate(model, test_dataloader)

        print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%")

def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = (correct / total) * 100
    return accuracy


In [36]:
# ...
num_epochs=10
train(model, train_dataloader, test_dataloader, criterion, optimizer, num_epochs)


TypeError: ignored

In [30]:
def predict(model, image):
    model.eval()
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
    return predicted.item()


In [ ]:
# Cargar el modelo previamente entrenado
model = cargar_modelo_entrenado()  # Reemplaza con la carga de tu modelo entrenado

# Ruta de la nueva imagen de basura
new_image_path = "ruta_de_la_nueva_imagen.jpg"  # Reemplaza con la ruta de la nueva imagen

# Leer la nueva imagen
new_image = Image.open(new_image_path)

# Realizar la predicción
predicted_label = predict(model, new_image)

# Obtener el nombre de la clase predicha
class_names = ['clase1', 'clase2', 'clase3']  # Reemplaza con los nombres de las clases de tu modelo
predicted_class = class_names[predicted_label]

# Mostrar la predicción
print("Predicción:", predicted_class)

In [ ]:
if __name__ == "__main__":
    # Paso 1: Configuración del entorno
    # Importar bibliotecas necesarias

    # Paso 3: Carga del dataset

    # Paso 4: Preprocesamiento de datos
    # Aplicar transformaciones a las imágenes

    # Paso 5: Creación del modelo
    # Cargar modelo preentrenado y modificar capa de clasificación

    # Paso 6: Entrenamiento del modelo
    # Crear dataloaders de entrenamiento y validación
    # Iterar sobre los datos de entrenamiento y realizar entrenamiento

    # Paso 7: Evaluación del modelo
    # Calcular la precisión del modelo en el conjunto de validación

    # Paso 8: Inferencia
    # Realizar predicciones en nuevas imágenes de basura

    # Otros pasos:
    # - Crear repositorio en GitHub y almacenar el cuadernillo y los recursos necesarios
    # - Almacenar el dataset en Google Drive y
